# Activation Vector

Method for calculating a Layer's Activation Vector based on Input (dataset) X

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.models import load_model
import sys
sys.path.append('PATH')
from utils import split_dataset, to_supervised, forecast

In [ ]:
def get_activation_vector(model, dataset, layer_number, save_path):
    # load model
    m = load_model(model)

    # define layer
    layer = m.layers[layer_number].output # e.g. 4
    # build layer model
    layer_model = Model(inputs=m.input,
                        outputs=layer)
    layer_model.summary()
    # set dataset variable
    #dataset = 'data/branch5.pkl'
    # load dataset
    df = pd.read_pickle(dataset)
    df = pd.DataFrame(data=df.values, index=df.index, columns=['netto'])
    # assign features for month, weekday, year
    df = df.assign(month=df.index.month)
    df = df.assign(weekday=df.index.weekday)
    df = df.assign(year=df.index.year)
    # split into train and test
    train, test = split_dataset(df.values, 365)
    # prepare input data for branch 2
    n_input = 365
    train_x, train_y = to_supervised(train, n_input, 365)

    # history is a list of yearly data
    history = [x for x in train]

    # walk-forward validation over each year (in our case we only predict 1 year!)
    prediction = list()
    for i in range(len(test)):
        # predict the year based on last years history
        yhat_sequence = forecast(layer_model, history, n_input)
        # store the predictions
        prediction.append(yhat_sequence)
        # get real observation and add to history for predicting the next year
        history.append(test[i, :])

    acts = np.array(prediction)
    np.savetxt(save_path + str(layer_number) + ".csv", acts[0], delimiter=",")
    del m, layer_model

In [ ]:
get_activation_vector('../models/pretrained/path.h5', '../data/preprocessed/path.pkl', 4, "activations/m5_x5/")

# Get Activation for Model5 on Input Datapoints Branch5 m5(x5)

In [ ]:
i = 4
while i < 32:
  get_activation_vector('../models/pretrained/path.h5', '../data/preprocessed/path.pkl', i, "activations/m5_x5/")
  i += 1

# Get Activation for Model6 on Input Datapoints Branch6 m6(x6)

In [ ]:
i = 4
while i < 32:
  get_activation_vector('../models/pretrained/path.h5', '../data/preprocessed/path.pkl', i, "activations/m6_x6/")
  i += 1

# m5(x5), m6(x5) two models, same input datapoints

In [ ]:
i = 4
while i < 32:
  get_activation_vector('../models/pretrained/path.h5', '../data/preprocessed/path.pkl', i, "activations/m6_x5/")
  i += 1